# Laser display boards

You need to navigate your way through Heathwick Airport to find the correct gate. The bad news is that all the display boards have gone down. The good news is that the terminal staff are handing out the machine-code instructions to generate the messages on the board. 

Given your l33t haxor skilz, it will be no problem to recreate the messages on the display boards.

The board is grid, 80 pixels wide and 8 pixels tall, with row 1 being the top row and column 1 being the left column. The pixels are changed with these commands:

* `top A B` switches the state of the pixels in the topmost row from columns A to B inclusive. If a pixel was lit, it becomes dark; if it was dark, it becomes lit.
* `left A B` is similar, but works on the left edge, toggling the state of pixles in the leftmost column in rows A to B inclusive. 
* `rotate column A B` rotates column A by B spaces down. Pixels that are moved beyond the bottom edge "wrap around" to the top edge.
* `rotate row A B` rotates row A by B spaces to the right. Pixels that are moved beyond the right edge "wrap around" to the left edge.

You can assume all numbers are integers, the row and column values are always valid, and `A` $\le$ `B` in the `left` and `toggle` commands.

For instance, with a smaller grid that is 10 pixels wide and 4 tall, this is what a sample sequence of instructions would do.

* `toggle 1 6` turns on the first six pixels on the top row.
```
******....
..........
..........
..........
```

* `rotate column 2 3` moves the lit pixel on the second column to the bottom row.
```
*.****....
..........
..........
.*........
```

* `toggle 3 10` turns off the pixels in columns 4, 5, and 6, and turns on the pixels in columns 7 to 10.

```
*.....****
..........
..........
.*........
```

* `rotate column 8 1` moves the one lit pixel in column 8 down one row.
```
*.....*.**
.......*..
..........
.*........
```

* `rotate row 2 6` moves that pixel off the right edge of the display, to it wraps around to appear in column 4.
```
*.....*.**
...*......
..........
.*........
```

* `left 1 3` toggles the pixels in rows 1, 2, and 3 of the first column. The top left pixel (previously on) turns off, while the pixels in rows 2 and 3 come on.
```
......*.**
*..*......
*.........
.*........
```

## Part 1
You're standing in front of gate 9¾. You have [the instructions](03-pixels.txt). How many pixels would be lit on the board, if it was working?

In [1]:
import time
import re
from IPython.display import clear_output

In [2]:
WIDTH = 80
HEIGHT = 8

In [3]:
def new_grid(w=WIDTH, h=HEIGHT):
    return ['.' * w for r in range(1, h+1)]

In [4]:
def print_grid(grid, md=False, suppress_dots=False):
    if md:
        print('```')
    for row in grid:
        if suppress_dots:
            print(re.sub(r'\.', ' ', row))
        else:
            print(row)
    if md:
        print('```')

In [5]:
def top(grid, l, r):
    new_segment = ''
    for i in range(l-1, r):
        if grid[0][i] == '.':
            new_segment += '*'
        else:
            new_segment += '.'
    grid[0] = grid[0][:l-1] + new_segment + grid[0][r:]
    return grid

In [6]:
def left(grid, t, b):
    for i in range(t-1, b):
        if grid[i][0] == '.':
            grid[i] = '*' + grid[i][1:]
        else:
            grid[i] = '.' + grid[i][1:]
    return grid

In [7]:
def rotate_column(grid, c, raw_n):
    n = raw_n % len(grid)
    col = [row[c-1] for row in grid]
    new_col = col[-n:] + col[:-n]
    for i in range(len(grid)):
        grid[i] = grid[i][:c-1] + new_col[i] + grid[i][c:]
    return grid

In [8]:
def rotate_row(grid, r, raw_n):
    n = raw_n % len(grid[0])
    grid[r-1] = grid[r-1][-n:] + grid[r-1][:-n]
    return grid

In [9]:
command_dispatch = {'left': left, 'top': top,
                   'rotate row': rotate_row,
                   'rotate column': rotate_column}

In [10]:
def parse(command):
    cmd, a, b = command.rsplit(maxsplit=2)
    return cmd, int(a), int(b)  

In [11]:
def interpret(commands, grid=None, w=WIDTH, h=HEIGHT, 
              show_each_step=False, md=False, overprint=False):
    if grid is None:
        grid = new_grid(w, h)
    for c in commands:
        cmd, a, b = parse(c)
        if cmd in command_dispatch:
            command_dispatch[cmd](grid, a, b)
        else:
            raise ValueError('Unknown command')
        if show_each_step:
            if overprint:
                time.sleep(0.25)
            if md: 
                print('`{}`'.format(c))
            else:
                print(c)
            print_grid(grid, md=md)
            print()
            if overprint:
                clear_output(wait=True)
    if show_each_step: 
        print('Final')
        print_grid(grid, md=md)
    return grid

In [12]:
cmds = [c.strip() for c in open('05-pixels.txt').readlines()]
len(cmds)

237

In [13]:
g = interpret(cmds)
sum(1 for c in ''.join(g) if c == '*')

188

## Part 2
Where does the flight go from gate 9¾?

In [14]:
print_grid(g)

...****..............*...................*.....*..............*.................
......*..............*...................***..**..............*.................
......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...
......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......
......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......
......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......
...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......
....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......


In [15]:
print_grid(g, suppress_dots=True)

   ****              *                   *     *              *                 
      *              *                   ***  **              *                 
      * ***** ***** **** *****  ****     * * *** ***** ***** **** *****  ****   
      *     * *   *  *       *  *        *  ** *     * *   *  *       *  *      
      * ***** *   *  *   *****  *        *  *  * ***** *   *  *   *****  *      
      * *   * *   *  *   *   *  *        *     * *   * *   *  *   *   *  *      
   *  * *  ** *   *  **  *  **  *        *     * *  ** *   *  **  *  **  *      
    **   ** * *   *   **  ** *  *        *     *  ** * *   *   **  ** *  *      
